In [ ]:
import random
import math
 
# Definición del área donde se pueden colocar las torres y sus parámetros
xmin, ymin = 0, 0         # Coordenadas mínimas del plano
xmax, ymax = 500, 500     # Coordenadas máximas del plano
rmax = 50                  # Radio máximo de cobertura para cada torre
numTorres = 10             # Número de torres a optimizar
 
def generar_torre():
    x = random.uniform(xmin, xmax)
    y = random.uniform(ymin, ymax)
    radio = random.uniform(1, rmax)
    costo = radio * 100  # Costo estimado según el radio de cobertura
    return {"x": x, "y": y, "radio": radio, "costo": costo}
 
 
def generar_lista_torres():
    return [generar_torre() for _ in range(numTorres)]
 
def torres_solapadas(torre1, torre2):
    distancia = math.sqrt((torre1['x'] - torre2['x']) ** 2 + (torre1['y'] - torre2['y']) ** 2)
    return distancia < (torre1['radio'] + torre2['radio'])
 
 
def calcular_fitness(lista_torres):
    cobertura_total = sum(torre['radio'] for torre in lista_torres)
    costo_total = sum(torre['costo'] for torre in lista_torres)
    # Penalización por solapamiento de torres
    penalizacion_solapamiento = 0
    for i, torre1 in enumerate(lista_torres):
        for torre2 in lista_torres[i+1:]:
            if torres_solapadas(torre1, torre2):
                penalizacion_solapamiento += 1
    # Fitness final: optimización de cobertura con penalización de costos y solapamiento
    fitness = cobertura_total - (costo_total * 0.5) - (penalizacion_solapamiento * 100)
    return fitness
 
 
class Particula:
 
    def __init__(self):
        self.posicion = generar_lista_torres()     # Posición inicial de la partícula (lista de torres)
        self.velocidad = [{"vx": 0, "vy": 0} for _ in range(numTorres)]  # Velocidad de cada torre
        self.best_posicion = self.posicion.copy()  # Mejor posición conocida
        self.best_fitness = calcular_fitness(self.posicion)
 
    def actualizar_velocidad(self, global_best, w=0.5, c1=1.5, c2=1.5):
        for i, torre in enumerate(self.posicion):
            vx = (
                w * self.velocidad[i]["vx"] 
                + c1 * random.random() * (self.best_posicion[i]['x'] - torre['x']) 
                + c2 * random.random() * (global_best[i]['x'] - torre['x'])
            )
            vy = (
                w * self.velocidad[i]["vy"] 
                + c1 * random.random() * (self.best_posicion[i]['y'] - torre['y']) 
                + c2 * random.random() * (global_best[i]['y'] - torre['y'])
            )
            self.velocidad[i] = {"vx": vx, "vy": vy}
            torre['x'] += vx
            torre['y'] += vy
 
    def actualizar_posicion(self):
        for torre in self.posicion:
            torre['x'] = max(xmin, min(xmax, torre['x']))
            torre['y'] = max(ymin, min(ymax, torre['y']))
            torre['radio'] = max(1, min(rmax, torre['radio']))
        fitness_actual = calcular_fitness(self.posicion)
        if fitness_actual > self.best_fitness:
            self.best_fitness = fitness_actual
            self.best_posicion = self.posicion.copy()
 
 
def optimizar_torres_pso(iteraciones=100, num_particulas=30):
    particulas = [Particula() for _ in range(num_particulas)]
    global_best = max(particulas, key=lambda p: p.best_fitness).best_posicion
 
    for iteracion in range(iteraciones):
        for particula in particulas:
            particula.actualizar_velocidad(global_best)
            particula.actualizar_posicion()
            if particula.best_fitness > calcular_fitness(global_best):
                global_best = particula.best_posicion
        # Imprime el progreso
        mejor_fitness = calcular_fitness(global_best)
        print(f"Iteración {iteracion + 1}: Mejor fitness encontrado = {mejor_fitness}")
 
    return global_best
 
 
mejor_solucion = optimizar_torres_pso()
print("Mejor solución encontrada:", mejor_solucion)